## পরিচিতি

এই পাঠে আলোচনা করা হবে:
- ফাংশন কল কী এবং এর ব্যবহার ক্ষেত্রসমূহ
- OpenAI ব্যবহার করে কীভাবে ফাংশন কল তৈরি করবেন
- কীভাবে একটি অ্যাপ্লিকেশনে ফাংশন কল সংযুক্ত করবেন

## শেখার লক্ষ্য

এই পাঠ শেষ করার পর আপনি জানতে ও বুঝতে পারবেন:

- ফাংশন কল ব্যবহারের উদ্দেশ্য
- OpenAI সার্ভিস ব্যবহার করে ফাংশন কল সেটআপ করা
- আপনার অ্যাপ্লিকেশনের জন্য কার্যকর ফাংশন কল ডিজাইন করা


## ফাংশন কল বোঝা

এই পাঠের জন্য, আমরা আমাদের এডুকেশন স্টার্টআপের জন্য এমন একটি ফিচার তৈরি করতে চাই, যা ব্যবহারকারীদের চ্যাটবট ব্যবহার করে টেকনিক্যাল কোর্স খুঁজে পেতে সাহায্য করবে। আমরা এমন কোর্স সাজেস্ট করব, যা তাদের দক্ষতার স্তর, বর্তমান পেশা এবং আগ্রহের প্রযুক্তির সাথে মানানসই।

এটি সম্পূর্ণ করতে আমরা নিচের কিছুর সমন্বয় ব্যবহার করব:
 - `OpenAI` ব্যবহারকারীর জন্য চ্যাট অভিজ্ঞতা তৈরি করতে
 - `Microsoft Learn Catalog API` ব্যবহারকারীর অনুরোধ অনুযায়ী কোর্স খুঁজে পেতে সাহায্য করতে
 - `Function Calling` ব্যবহারকারীর প্রশ্ন নিয়ে সেটি ফাংশনে পাঠিয়ে API অনুরোধ করতে

শুরু করার জন্য, চলুন দেখি আমরা প্রথমেই কেন ফাংশন কল ব্যবহার করতে চাই:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # GPT থেকে নতুন রেসপন্স পাওয়া, যেখানে এটি ফাংশনের রেসপন্স দেখতে পারে


print(second_response.choices[0].message)


### কেন ফাংশন কলিং

আপনি যদি এই কোর্সের অন্য কোনো পাঠ সম্পন্ন করে থাকেন, তাহলে সম্ভবত আপনি বড় ভাষা মডেল (LLM) ব্যবহারের শক্তি বুঝতে পেরেছেন। আশা করি আপনি তাদের কিছু সীমাবদ্ধতাও দেখতে পেয়েছেন।

ফাংশন কলিং হলো OpenAI সার্ভিসের একটি ফিচার, যা নিচের চ্যালেঞ্জগুলো সমাধানের জন্য তৈরি করা হয়েছে:

অসংগতিপূর্ণ রেসপন্স ফরম্যাটিং:
- ফাংশন কলিংয়ের আগে, বড় ভাষা মডেল থেকে পাওয়া রেসপন্সগুলো ছিল অগোছালো এবং অসংগতিপূর্ণ। ডেভেলপারদেরকে আউটপুটের প্রতিটি ভিন্নতার জন্য জটিল ভ্যালিডেশন কোড লিখতে হতো।

বহিঃস্থ ডেটার সাথে সীমিত সংযোগ:
- এই ফিচারটি আসার আগে, কোনো অ্যাপ্লিকেশনের অন্য অংশ থেকে ডেটা চ্যাট কনটেক্সটে আনা বেশ কঠিন ছিল।

রেসপন্স ফরম্যাটকে স্ট্যান্ডার্ড করে এবং বহিঃস্থ ডেটার সাথে সহজ সংযোগের সুযোগ দিয়ে, ফাংশন কলিং ডেভেলপমেন্টকে সহজ করে এবং অতিরিক্ত ভ্যালিডেশন লজিকের প্রয়োজন কমিয়ে দেয়।

ব্যবহারকারীরা "স্টকহোমে এখনকার আবহাওয়া কেমন?"—এমন প্রশ্নের উত্তর পেতেন না। কারণ, মডেলগুলো শুধুমাত্র তাদের প্রশিক্ষণের সময় পর্যন্ত ডেটা জানত।

চলুন নিচের উদাহরণটি দেখি, যা এই সমস্যাটি বোঝায়:

ধরা যাক, আমরা শিক্ষার্থীদের ডেটাবেস তৈরি করতে চাই, যাতে তাদের জন্য সঠিক কোর্স সাজেস্ট করতে পারি। নিচে আমরা দুইজন শিক্ষার্থীর বর্ণনা দিয়েছি, যাদের ডেটা খুবই মিল রয়েছে।


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

আমরা চাই এটি একটি LLM-এ পাঠাতে যাতে ডেটা পার্স করা যায়। পরে এটি আমাদের অ্যাপ্লিকেশনে API-তে পাঠানো বা ডাটাবেজে সংরক্ষণ করার জন্য ব্যবহার করা যেতে পারে।

চলুন দুটি একদম একই রকম প্রম্পট তৈরি করি, যেখানে আমরা LLM-কে নির্দেশনা দেবো কোন কোন তথ্য আমাদের দরকার:


আমরা এটি একটি এলএলএম-এ পাঠাতে চাই যাতে এটি আমাদের পণ্যের জন্য গুরুত্বপূর্ণ অংশগুলি বিশ্লেষণ করতে পারে। তাই আমরা এলএলএম-কে নির্দেশনা দেওয়ার জন্য দুটি অভিন্ন প্রম্পট তৈরি করতে পারি:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


এই দুটি প্রম্পট তৈরি করার পরে, আমরা সেগুলোকে LLM-এ পাঠাব `openai.ChatCompletion` ব্যবহার করে। আমরা প্রম্পটটি `messages` ভ্যারিয়েবলে সংরক্ষণ করি এবং রোলটি `user` হিসেবে নির্ধারণ করি। এটি একটি চ্যাটবটের কাছে ব্যবহারকারীর বার্তা পাঠানোর অনুকরণ করার জন্য।


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

এখন আমরা উভয় অনুরোধ LLM-এ পাঠাতে পারি এবং আমরা যে প্রতিক্রিয়া পাই তা পরীক্ষা করতে পারি।


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

যদিও প্রম্পটগুলো একই এবং বর্ণনাগুলোও প্রায় একই রকম, আমরা `Grades` প্রপার্টির বিভিন্ন ফরম্যাট পেতে পারি।

আপনি যদি উপরের সেলটি একাধিকবার চালান, তাহলে ফরম্যাট হতে পারে `3.7` অথবা `3.7 GPA`।

এর কারণ হলো LLM লিখিত প্রম্পটের আকারে অসংগঠিত ডেটা নেয় এবং অসংগঠিত ডেটাই ফেরত দেয়। আমাদের একটি সংগঠিত ফরম্যাট দরকার, যাতে আমরা জানি এই ডেটা সংরক্ষণ বা ব্যবহারের সময় কী আশা করা উচিত।

ফাংশনাল কলিং ব্যবহার করলে, আমরা নিশ্চিত হতে পারি যে আমরা সংগঠিত ডেটা ফেরত পাচ্ছি। ফাংশন কলিং ব্যবহারের সময়, LLM আসলে কোনো ফাংশন কল বা চালায় না। বরং, আমরা LLM-এর জন্য একটি কাঠামো তৈরি করি, যাতে সে তার উত্তরগুলো সেই কাঠামো অনুসরণ করে দেয়। এরপর আমরা সেই সংগঠিত উত্তরগুলো ব্যবহার করি, আমাদের অ্যাপ্লিকেশনে কোন ফাংশন চালাতে হবে তা নির্ধারণ করতে।


![ফাংশন কলিং ফ্লো ডায়াগ্রাম](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.bn.png)


তারপর আমরা ফাংশন থেকে যা ফেরত আসে তা নিতে পারি এবং এটি আবার LLM-এ পাঠাতে পারি। এরপর LLM প্রাকৃতিক ভাষা ব্যবহার করে ব্যবহারকারীর প্রশ্নের উত্তর দেবে।


### ফাংশন কল ব্যবহারের ব্যবহারিক ক্ষেত্রসমূহ

**বাহ্যিক টুল ব্যবহারের জন্য কল করা**
চ্যাটবটগুলো ব্যবহারকারীদের প্রশ্নের উত্তর দিতে দারুণ দক্ষ। ফাংশন কলিং ব্যবহার করে, চ্যাটবটগুলো ব্যবহারকারীর বার্তা থেকে নির্দিষ্ট কিছু কাজ সম্পন্ন করতে পারে। উদাহরণস্বরূপ, একজন শিক্ষার্থী চ্যাটবটকে বলতে পারে, "আমার শিক্ষককে ইমেইল পাঠাও যে আমি এই বিষয়ে আরও সহায়তা চাই।" এটি `send_email(to: string, body: string)` ফাংশনটি কল করতে পারে।

**API বা ডাটাবেস কোয়েরি তৈরি করা**
ব্যবহারকারীরা স্বাভাবিক ভাষায় তথ্য খুঁজে পেতে পারে, যা একটি ফরম্যাটেড কোয়েরি বা API অনুরোধে রূপান্তরিত হয়। এর একটি উদাহরণ হতে পারে, একজন শিক্ষক জিজ্ঞাসা করছেন, "শেষ অ্যাসাইনমেন্টটি কে কে সম্পন্ন করেছে?"—এটি `get_completed(student_name: string, assignment: int, current_status: string)` নামের একটি ফাংশন কল করতে পারে।

**স্ট্রাকচার্ড ডেটা তৈরি করা**
ব্যবহারকারীরা একটি টেক্সট ব্লক বা CSV নিয়ে LLM ব্যবহার করে সেখান থেকে গুরুত্বপূর্ণ তথ্য বের করতে পারে। উদাহরণস্বরূপ, একজন শিক্ষার্থী শান্তি চুক্তি নিয়ে উইকিপিডিয়ার একটি আর্টিকেলকে AI ফ্ল্যাশ কার্ডে রূপান্তর করতে পারে। এটি `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` নামের একটি ফাংশন ব্যবহার করে করা যেতে পারে।


## ২. আপনার প্রথম ফাংশন কল তৈরি করা

একটি ফাংশন কল তৈরির প্রক্রিয়ায় ৩টি প্রধান ধাপ রয়েছে:
১. আপনার ফাংশনগুলোর একটি তালিকা এবং ব্যবহারকারীর বার্তা সহ Chat Completions API-তে কল করা
২. মডেলের উত্তর পড়ে একটি অ্যাকশন নেওয়া, অর্থাৎ কোনো ফাংশন বা API কল এক্সিকিউট করা
৩. আপনার ফাংশনের উত্তর নিয়ে আবার Chat Completions API-তে কল করা, যাতে সেই তথ্য ব্যবহার করে ব্যবহারকারীর জন্য একটি উত্তর তৈরি করা যায়।


![Flow of a Function Call](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.bn.png)


### একটি ফাংশন কলের উপাদানসমূহ

#### ব্যবহারকারীর ইনপুট

প্রথম ধাপ হলো একটি ব্যবহারকারীর বার্তা তৈরি করা। এটি আপনি টেক্সট ইনপুট থেকে ডাইনামিকভাবে নিতে পারেন অথবা এখানে একটি মান নির্ধারণ করতে পারেন। যদি আপনি প্রথমবারের মতো Chat Completions API নিয়ে কাজ করেন, তাহলে আমাদের বার্তার `role` এবং `content` নির্ধারণ করতে হবে।

`role` হতে পারে `system` (নিয়ম তৈরি করা), `assistant` (মডেল) অথবা `user` (চূড়ান্ত ব্যবহারকারী)। ফাংশন কলের জন্য, আমরা এটিকে `user` হিসেবে নির্ধারণ করব এবং একটি উদাহরণ প্রশ্ন দেব।


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### ফাংশন তৈরি করা

এবার আমরা একটি ফাংশন এবং তার প্যারামিটার নির্ধারণ করব। এখানে আমরা শুধু একটি ফাংশন ব্যবহার করব যার নাম `search_courses`, তবে আপনি চাইলে একাধিক ফাংশন তৈরি করতে পারেন।

**গুরুত্বপূর্ণ** : ফাংশনগুলো LLM-এর সিস্টেম মেসেজে যুক্ত হয় এবং এগুলো আপনার ব্যবহারের জন্য উপলব্ধ টোকেনের পরিমাণে গণনা হয়।


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**সংজ্ঞাসমূহ**

ফাংশন ডেফিনিশনের গঠন একাধিক স্তরে বিভক্ত, প্রতিটি স্তরের নিজস্ব বৈশিষ্ট্য রয়েছে। এখানে নেস্টেড গঠনের একটি সংক্ষিপ্ত বিবরণ দেওয়া হলো:

**শীর্ষ স্তরের ফাংশন বৈশিষ্ট্যাবলী:**

`name` -  ফাংশনের নাম যেটি আমরা কল করতে চাই।

`description` - এটি ফাংশনটি কীভাবে কাজ করে তার বর্ণনা। এখানে নির্দিষ্ট ও স্পষ্ট হওয়া গুরুত্বপূর্ণ।

`parameters` - এমন মান ও ফরম্যাটের একটি তালিকা, যা আপনি চান মডেল তার উত্তরে তৈরি করুক।

**প্যারামিটার অবজেক্টের বৈশিষ্ট্যাবলী:**

`type` -  প্যারামিটার অবজেক্টের ডেটা টাইপ (সাধারণত "object")

`properties` - নির্দিষ্ট মানগুলোর তালিকা, যেগুলো মডেল তার উত্তরে ব্যবহার করবে

**প্রতিটি প্যারামিটারের বৈশিষ্ট্যাবলী:**

`name` - প্রপার্টি কী দ্বারা পরোক্ষভাবে নির্ধারিত (যেমন, "role", "product", "level")

`type` - এই নির্দিষ্ট প্যারামিটারটির ডেটা টাইপ (যেমন, "string", "number", "boolean")

`description` - নির্দিষ্ট প্যারামিটারটির বর্ণনা

**ঐচ্ছিক বৈশিষ্ট্যাবলী:**

`required` - একটি অ্যারে, যেখানে তালিকাভুক্ত থাকে কোন কোন প্যারামিটার ফাংশন কল সম্পন্ন করতে আবশ্যক


### ফাংশন কল করা
একটি ফাংশন সংজ্ঞায়িত করার পর, এখন আমাদের এটি Chat Completion API-তে কল করার সময় অন্তর্ভুক্ত করতে হবে। আমরা এটি করি `functions` রিকোয়েস্টে যোগ করে। এই ক্ষেত্রে `functions=functions`।

এছাড়াও, `function_call`-কে `auto` হিসেবে সেট করার অপশন আছে। এর মানে হচ্ছে, আমরা LLM-কে সিদ্ধান্ত নিতে দেবো কোন ফাংশনটি কল করা উচিত, ব্যবহারকারীর মেসেজের উপর ভিত্তি করে, নিজেরা নির্ধারণ না করে।


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

এখন চলুন আমরা প্রতিক্রিয়াটি দেখি এবং এটি কীভাবে ফরম্যাট করা হয়েছে তা দেখি:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

আপনি দেখতে পাচ্ছেন, এখানে ফাংশনের নাম ডাকা হয়েছে এবং ব্যবহারকারীর বার্তা থেকে, LLM ফাংশনের আর্গুমেন্টে মান বসাতে পেরেছে।


## ৩. অ্যাপ্লিকেশনে ফাংশন কল ইন্টিগ্রেশন করা

আমরা যখন LLM থেকে ফরম্যাট করা রেসপন্স টেস্ট করে নিয়েছি, তখন এখন এটি আমাদের অ্যাপ্লিকেশনে ইন্টিগ্রেট করতে পারি।

### ফ্লো ম্যানেজ করা

এটি আমাদের অ্যাপ্লিকেশনে ইন্টিগ্রেট করতে, চলুন নিচের ধাপগুলো অনুসরণ করি:

প্রথমে, OpenAI সার্ভিসে কল করি এবং মেসেজটি `response_message` নামক একটি ভ্যারিয়েবলে সংরক্ষণ করি।


In [ ]:
response_message = response.choices[0].message

এখন আমরা সেই ফাংশনটি সংজ্ঞায়িত করব যা মাইক্রোসফট লার্ন এপিআই-তে কল করবে কোর্সের একটি তালিকা পাওয়ার জন্য:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



সেরা অনুশীলন হিসেবে, আমরা প্রথমে দেখব মডেলটি কোনো ফাংশন কল করতে চায় কিনা। এরপর, আমরা উপলব্ধ ফাংশনগুলোর একটি তৈরি করব এবং সেটিকে সেই ফাংশনের সাথে মিলিয়ে দেখব যা কল করা হচ্ছে।
এরপর, আমরা ফাংশনের আর্গুমেন্টগুলো নেব এবং সেগুলো LLM থেকে পাওয়া আর্গুমেন্টের সাথে মিলিয়ে দেখব।

সবশেষে, আমরা ফাংশন কলের বার্তা এবং `search_courses` বার্তা থেকে পাওয়া মানগুলো যোগ করব। এতে LLM-এর কাছে ব্যবহারকারীকে স্বাভাবিক ভাষায় উত্তর দেওয়ার জন্য প্রয়োজনীয় সব তথ্য থাকবে।


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



এখন আমরা আপডেট করা বার্তাটি LLM-এ পাঠাবো যাতে আমরা API JSON ফরম্যাটেড প্রতিক্রিয়ার পরিবর্তে একটি স্বাভাবিক ভাষার প্রতিক্রিয়া পেতে পারি।


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## কোড চ্যালেঞ্জ

দারুণ কাজ! OpenAI Function Calling শেখার জন্য আপনি আরও এগিয়ে যেতে পারেন: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - ফাংশনের আরও কিছু প্যারামিটার যোগ করুন, যা শিক্ষার্থীদের আরও কোর্স খুঁজে পেতে সাহায্য করতে পারে। এখানে আপনি API-র উপলব্ধ প্যারামিটারগুলো দেখতে পাবেন:  
 - আরেকটি ফাংশন কল তৈরি করুন, যেখানে শিক্ষার্থীর মাতৃভাষার মতো আরও তথ্য নেওয়া হবে  
 - ফাংশন কল এবং/অথবা API কল থেকে যদি কোনো উপযুক্ত কোর্স না ফেরত আসে, তাহলে এর জন্য ত্রুটি ব্যবস্থাপনা তৈরি করুন



---

**দায়িত্ব অস্বীকার**:
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব নির্ভুল অনুবাদের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন, স্বয়ংক্রিয় অনুবাদে ভুল বা অসঙ্গতি থাকতে পারে। মূল ভাষায় রচিত নথিটিকেই চূড়ান্ত ও নির্ভরযোগ্য উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য পেশাদার মানব অনুবাদ গ্রহণ করার পরামর্শ দেওয়া হচ্ছে। এই অনুবাদের ব্যবহারে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়ী থাকব না।
